# Keras to Tensorflow Tutorial
This notebook show you how to train a model in Keras, deploy a freeze Tensorflow model and train on it

In [ ]:
import sys
sys.path.insert('../../util')
from keras_to_tensorflow import save_frozen_tf_model
from keras.models import Model

## Get mnist dataset

In [ ]:
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import RMSprop

from keras.utils import np_utils
import matplotlib.pyplot as plt

# download the mnist to the path '~/.keras/datasets/' if it is the first time to be called
# X shape (60,000 28x28), y shape (10,000, )
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# data pre-processing
X_train = X_train.reshape(X_train.shape[0], -1) / 255.   # normalize
X_test = X_test.reshape(X_test.shape[0], -1) / 255.      # normalize
y_train = np_utils.to_categorical(y_train, num_classes=10)    #one hot
y_test = np_utils.to_categorical(y_test, num_classes=10)

## Train convolutional neural network on Keras

In [ ]:
# Create model
model = Sequential()
model.add(Dense(32, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax', name='pred'))

# Define optimizer
rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

# We add metrics to get more results you want to see
model.compile(optimizer=rmsprop,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print('Training ------------')
# Another way to train the model
model.fit(X_train, y_train, epochs=2, batch_size=256)

## Freeze the network and write to protobuf format

In [ ]:
save_frozen_tf_model(model, 'model.pb')

## Load Tensorflow model

In [ ]:
import tensorflow as tf

with tf.gfile.GFile('model.pb', "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

with tf.Graph().as_default() as graph:
    tf.import_graph_def(graph_def, name='')
    
for op in graph.get_operations():
    print(op.name)

## Compare result of Tensorflow and Keras

In [ ]:
# predict from tensorflow model (freeze)
x = graph.get_tensor_by_name('dense_1_input:0')
y = graph.get_tensor_by_name('pred/Softmax:0')

with tf.Session(graph=graph) as sess:
    # Note: we don't need to initialize/restore anything
    # There is no Variables in this graph, only hardcoded constants   
    digit = X_test[np.random.randint(10000)]
    img = digit.reshape(28, 28)
    plt.imshow(img, cmap='gray')
    plt.show()

    y_tf_out = sess.run(y, feed_dict={x: digit.reshape(1, 784)})
    plt.bar(range(0, 10), y_tf_out[0])
    plt.show()

In [ ]:
# predict from keras model
img = digit.reshape(28, 28)
plt.imshow(img, cmap='gray')
plt.show()

y_keras_out = model.predict(digit.reshape(1, 784))
plt.bar(range(0, 10), y_keras_out[0])
plt.show()